# Python で気軽に化学・化学工学
# 第 8 章 モデル y = f(x) を構築して、新たなサンプルの y を推定する
## 8.7.3 決定木 (Decision Tree, DT)
### クラス分類

## Jupyter Notebook の有用なショートカットのまとめ
- <kbd>Esc</kbd>: コマンドモードに移行（セルの枠が青）
- <kbd>Enter</kbd>: 編集モードに移行（セルの枠が緑）
- コマンドモードで <kbd>M</kbd>: Markdown セル (説明・メモを書く用) に変更
- コマンドモードで <kbd>Y</kbd>: Code セル (Python コードを書く用) に変更
- コマンドモードで <kbd>H</kbd>: ヘルプを表示
- コマンドモードで <kbd>A</kbd>: ひとつ**上**に空のセルを挿入
- コマンドモードで <kbd>B</kbd>: ひとつ**下**に空のセルを挿入
- コマンドモードで <kbd>D</kbd><kbd>D</kbd>: セルを削除
- <kbd>Ctrl</kbd>+<kbd>Enter</kbd>: セルの内容を実行
- <kbd>Shift</kbd>+<kbd>Enter</kbd>: セルの内容を実行して下へ

### あやめのデータセット (iris_with_species.csv)
有名な [Fisher’s Iris Data](https://en.wikipedia.org/wiki/Iris_flower_data_set)。150個のあやめについて、がく片長(Sepal Length)、がく片幅(Sepal Width)、花びら長(Petal Length)、花びら幅(Petal Width)が計測されています。

In [ ]:
import pandas as pd # pandas のインポート

In [ ]:
dataset = pd.read_csv('iris_with_species.csv', index_col=0, header=0) # あやめのデータセットの読み込み

DT は SVM とは異なり、3 つのクラスがあっても問題ありません。SVM のように setosa と versicolor で 1 つのクラス `setosa+versicolor` にはまとめずに、setosa, versicolor, virginica の 3 つのクラスを分類します。

In [ ]:
# y と x に分割
y = dataset.iloc[:,0]
x = dataset.iloc[:,1:]

トレーニングデータとテストデータの分割

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# ランダムにトレーニングデータとテストデータとに分割。random_state に数字を与えることで、別のときに同じ数字を使えば、ランダムとはいえ同じ結果にすることができます
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=50, stratify=y, shuffle=True, random_state=3)

DT モデルにおいては、特徴量 x の値が計算に用いられることはなく、x の値が何らかの閾値より大きいか小さいかのみが必要です。そこで、一般的に DT モデルを構築するときは、x の標準化  (オートスケーリング) は行いません。

DT の実行

In [ ]:
from sklearn.tree import DecisionTreeClassifier # クラス分類用の DT の実行に使用

In [ ]:
model = DecisionTreeClassifier(max_depth=3, min_samples_leaf=3) # DTモデルの宣言

In [ ]:
model.fit(x_train, y_train) # DTモデル構築

構築された DT モデルの確認

DT モデルは木のような構造で与えられます。その内容を dot ファイル (拡張子が dot であるテキストファイル) に保存します。別途、Graphviz というソフトウェア (無料) を用いて dot ファイルを開けば、DT モデルの内容を可視化できます。Graphviz は [こちら](https://www.graphviz.org/download/) からダウンロードできます。

In [ ]:
from sklearn.tree import export_graphviz # dot ファイル作成に使用

dotファイルの作成。`tree.dot` というファイルが作成されます。

In [ ]:
with open('tree.dot', 'w') as f:
    if model.classes_.dtype == 'object':
        classnames = model.classes_
    else:
        # クラス名が数値のときの対応
        classnames = []
        for classnamenumber in range(0,model.classes_.shape[0]):
            classnames.append( str(model.classes_[classnamenumber]) )
    export_graphviz(model, out_file=f, feature_names=x.columns, class_names=classnames)

`tree.dot` を Graphviz で開くと木の内容を確認できます。

トレーニングデータのクラスの推定

In [ ]:
estimated_y_train = pd.DataFrame(model.predict(x_train), index=x_train.index, columns=['estimated_class']) # 推定し、pandas の DataFrame 型に変換

In [ ]:
estimated_y_train.to_csv('estimated_y_train.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

トレーニングデータの混同行列

In [ ]:
from sklearn import metrics # 混同行列の作成、正解率の計算に使用

In [ ]:
class_types = list(set(y_train)) # リスト型に変換。これで混同行列における縦と横のクラスの順番を定めます

In [ ]:
class_types.sort() # アルファベット順に並び替え

In [ ]:
confusion_matrix_train = pd.DataFrame(metrics.confusion_matrix(y_train, estimated_y_train, labels=class_types)) # 混同行列を作成し、pandas の DataFrame 型に変換

In [ ]:
confusion_matrix_train.index = class_types # 行の名前を、定めたクラスの名前に
confusion_matrix_train.columns = class_types # 列の名前、定めたクラスの名前に

In [ ]:
confusion_matrix_train # 確認

In [ ]:
confusion_matrix_train.to_csv('confusion_matrix_train.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

In [ ]:
metrics.accuracy_score(y_train, estimated_y_train) # 正解率

テストデータのクラスの推定。トレーニングデータをテストデータに変えるだけで、実行する内容はトレーニングデータのときと同じです

In [ ]:
estimated_y_test = pd.DataFrame(model.predict(x_test), index=x_test.index, columns=['estimated_class']) # 推定し、pandas の DataFrame 型に変換

In [ ]:
estimated_y_test # 念のため確認

In [ ]:
estimated_y_test.to_csv('estimated_y_test.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

テストデータの混同行列

In [ ]:
confusion_matrix_test = pd.DataFrame(metrics.confusion_matrix(y_test, estimated_y_test, labels=class_types)) # 混同行列を作成し、pandas の DataFrame 型に変換

In [ ]:
confusion_matrix_test.index = class_types # 行の名前を、定めたクラスの名前に
confusion_matrix_test.columns = class_types # 列の名前、定めたクラスの名前に

In [ ]:
confusion_matrix_test # 確認

In [ ]:
confusion_matrix_test.to_csv('confusion_matrix_test.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

In [ ]:
metrics.accuracy_score(y_test, estimated_y_test) # 正解率

クロスバリデーションによる木の深さの最大値の最適化

In [ ]:
import numpy as np # NumPy のインポート

In [ ]:
max_depths = np.arange(1, 31) # 木の深さの最大値の候補

In [ ]:
max_depths # 念のため確認

クロスバリデーションの設定

In [ ]:
fold_number = 10 # クロスバリデーションのfold数

In [ ]:
from sklearn.model_selection import StratifiedKFold # クロスバリデーションの分割の設定に使用

In [ ]:
fold = StratifiedKFold(n_splits=fold_number, shuffle=True, random_state=9) # クロスバリデーションの分割の設定。(KFold ではなく) StratifiedKFold を使用することで、fold ごとのクラスの割合がなるべく同じになるように分割されます

クロスバリデーションの実行

In [ ]:
from sklearn.model_selection import cross_val_predict # クロスバリデーションに使用

In [ ]:
accuracy_cv = [] # 空の list。木の深さの最大値ごとに、クロスバリデーション後の正解率を入れていきます

In [ ]:
for max_depth in max_depths:
    model = DecisionTreeClassifier(max_depth=max_depth, min_samples_leaf=3)
    estimated_y_in_cv = cross_val_predict(model, x_train, y_train, cv=fold)
    accuracy_cv.append(metrics.accuracy_score(y_train, estimated_y_in_cv))

In [ ]:
import matplotlib.pyplot as plt # 図の描画に使用

In [ ]:
# 結果の確認
plt.rcParams['font.size'] = 18
plt.scatter(max_depths, accuracy_cv)
plt.xlabel('maximum depth of tree')
plt.ylabel('accuracy in cross-validation')
plt.show()

In [ ]:
optimal_max_depth = max_depths[accuracy_cv.index(max(accuracy_cv))] # クロスバリデーション正解率が最大となる木の深さ

In [ ]:
optimal_max_depth # 念のため確認

DT モデルの構築および予測

In [ ]:
model = DecisionTreeClassifier(max_depth=optimal_max_depth, min_samples_leaf=3) # DTモデルの宣言

In [ ]:
model.fit(x_train, y_train) # DTモデル構築

構築された DT モデルの確認

In [ ]:
from sklearn.tree import export_graphviz # dot ファイル作成に使用

In [ ]:
with open('tree.dot', 'w') as f:
    if model.classes_.dtype == 'object':
        classnames = model.classes_
    else:
        # クラス名が数値のときの対応
        classnames = []
        for classnamenumber in range(0,model.classes_.shape[0]):
            classnames.append( str(model.classes_[classnamenumber]) )
    export_graphviz(model, out_file=f, feature_names=x.columns, class_names=classnames)

`tree.dot` を Graphviz で開くと木の内容を確認できます。

トレーニングデータのクラスの推定

In [ ]:
estimated_y_train = pd.DataFrame(model.predict(x_train), index=x_train.index, columns=['estimated_class']) # 推定し、pandas の DataFrame 型に変換

In [ ]:
estimated_y_train.to_csv('estimated_y_train.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

トレーニングデータの混同行列

In [ ]:
from sklearn import metrics # 混同行列の作成、正解率の計算に使用

In [ ]:
class_types = list(set(y_train)) # リスト型に変換。これで混同行列における縦と横のクラスの順番を定めます

In [ ]:
class_types.sort() # アルファベット順に並び替え

In [ ]:
confusion_matrix_train = pd.DataFrame(metrics.confusion_matrix(y_train, estimated_y_train, labels=class_types)) # 混同行列を作成し、pandas の DataFrame 型に変換

In [ ]:
confusion_matrix_train.index = class_types # 行の名前を、定めたクラスの名前に
confusion_matrix_train.columns = class_types # 列の名前、定めたクラスの名前に

In [ ]:
confusion_matrix_train # 確認

In [ ]:
confusion_matrix_train.to_csv('confusion_matrix_train.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

In [ ]:
metrics.accuracy_score(y_train, estimated_y_train) # 正解率

テストデータのクラスの推定。トレーニングデータをテストデータに変えるだけで、実行する内容はトレーニングデータのときと同じです

In [ ]:
estimated_y_test = pd.DataFrame(model.predict(x_test), index=x_test.index, columns=['estimated_class']) # 推定し、pandas の DataFrame 型に変換

In [ ]:
estimated_y_test # 念のため確認

In [ ]:
estimated_y_test.to_csv('estimated_y_test.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

テストデータの混同行列

In [ ]:
confusion_matrix_test = pd.DataFrame(metrics.confusion_matrix(y_test, estimated_y_test, labels=class_types)) # 混同行列を作成し、pandas の DataFrame 型に変換

In [ ]:
confusion_matrix_test.index = class_types # 行の名前を、定めたクラスの名前に
confusion_matrix_test.columns = class_types # 列の名前、定めたクラスの名前に

In [ ]:
confusion_matrix_test # 確認

In [ ]:
confusion_matrix_test.to_csv('confusion_matrix_test.csv') # csv ファイルに保存。同じ名前のファイルがあるときは上書きされますので注意してください

In [ ]:
metrics.accuracy_score(y_test, estimated_y_test) # 正解率

自分のデータセットをお持ちの方は、そのデータセットでも今回の内容を確認してみましょう。